In [48]:
from torch_geometric.datasets import planetoid
from torch_geometric.loader import DataLoader
import torch_geometric as pyg

In [49]:
dataset = planetoid.Planetoid(root='data/', name='Cora', split="public")
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [50]:
print(dataset.x.shape)
print(dataset.y.shape)
print(dataset.edge_index.shape)

torch.Size([2708, 1433])
torch.Size([2708])
torch.Size([2, 10556])


In [51]:
print(dataset.y.unique(return_counts=True))

(tensor([0, 1, 2, 3, 4, 5, 6]), tensor([351, 217, 418, 818, 426, 298, 180]))


In [52]:
print(dataset.train_mask.sum())
print(dataset.val_mask.sum())
print(dataset.test_mask.sum())

tensor(140)
tensor(500)
tensor(1000)


In [53]:
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [59]:
from torch_geometric.utils import subgraph
import torch

num_nodes = data.num_nodes
sample_size = num_nodes // 10  # 1/10 of the nodes
data.A = pyg.utils.to_dense_adj(data.edge_index)[0]
sampled_nodes = torch.randperm(num_nodes)[:sample_size]
subgraph_edge_index, subgraph_edge_attr = subgraph(sampled_nodes, data.edge_index, data.edge_attr)
subgraph_data = data.clone()
subgraph_data.edge_index = subgraph_edge_index
subgraph_data.edge_attr = subgraph_edge_attr
subgraph_data.x = data.x[sampled_nodes]
subgraph_data.y = data.y[sampled_nodes]
subgraph_data.train_mask = data.train_mask[sampled_nodes]
subgraph_data.val_mask = data.val_mask[sampled_nodes]
subgraph_data.test_mask = data.test_mask[sampled_nodes]
subgraph_data.A = pyg.utils.to_dense_adj(data.edge_index)[:, sampled_nodes, :][:, :, sampled_nodes]
print(subgraph_data.A.shape)

torch.Size([1, 270, 270])


In [63]:
dataset.data

/cis/home/yma93/miniconda3/envs/ml/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [55]:
pyg.utils.to_dense_adj(data.edge_index).shape

torch.Size([1, 2708, 2708])

In [47]:

A = pyg.utils.to_dense_adj(subgraph_data.edge_index)
degree = A.sum(dim=1).squeeze(dim=0)
print((degree>0).sum())

tensor(75)


In [42]:
print(subgraph_data)
print(subgraph_data.train_mask.sum())
print(subgraph_data.val_mask.sum())
print(subgraph_data.test_mask.sum())

Data(x=[270, 1433], edge_index=[2, 96], y=[270], train_mask=[270], val_mask=[270], test_mask=[270])
tensor(10)
tensor(53)
tensor(105)


In [4]:
import torch

A = torch.randn(1,3,3)
X = torch.randn(1,3,2)
print(A)
print(X)
n = A.shape[-1]  # extract dimension
        
# Compute basis
A = A.unsqueeze(dim=1) # N x 1 x n x n
X = X.transpose(-2, -1) # N x D1 x n
diag_part = torch.diagonal(A, dim1=-2, dim2=-1)   # N x 1 x n
mean_diag_part = torch.mean(diag_part, dim=-1).unsqueeze(dim=-1)  # N x 1 x 1
mean_of_cols = torch.mean(A, dim=-1)  # N x 1 x n
mean_all = torch.mean(mean_of_cols, dim=-1).unsqueeze(dim=-1)  # N x 1 x 1
mean_X = torch.mean(X, dim=-1).unsqueeze(dim=-1) #N x D1 x 1
a1 = A
a2 = mean_all.unsqueeze(dim=-1).expand(-1, -1, n, n)
a3 = mean_diag_part.unsqueeze(dim=-1).expand(-1, -1, n, n)
a4 = mean_of_cols.unsqueeze(dim=-1).expand(-1, -1, n, n) + mean_of_cols.unsqueeze(dim=-2).expand(-1, -1, n, n)
a5 = diag_part.unsqueeze(dim=-1).expand(-1, -1, n, n) + diag_part.unsqueeze(dim=-2).expand(-1, -1, n, n)
a6 = X.unsqueeze(dim=-1).expand(-1, -1, n, n) + X.unsqueeze(dim=-2).expand(-1, -1, n, n) # N x D1 x n x n
a7 = mean_X.unsqueeze(dim=-1).expand(-1, -1, n, n) # N x D1 x n x n

tensor([[[-2.4455,  1.0693, -1.5900],
         [ 1.3882,  0.1906,  0.7937],
         [-0.5306,  0.7166, -0.5505]]])
tensor([[[-0.5627, -1.5600],
         [-1.5452,  0.0567],
         [ 2.1319, -1.7710]]])


In [12]:
a6

tensor([[[[-1.1254, -2.1079,  1.5692],
          [-2.1079, -3.0905,  0.5867],
          [ 1.5692,  0.5867,  4.2638]],

         [[-3.1201, -1.5033, -3.3310],
          [-1.5033,  0.1134, -1.7143],
          [-3.3310, -1.7143, -3.5420]]]])

In [10]:
mean_of_cols

tensor([[[-0.9887,  0.7908, -0.1215]]])

In [3]:
import data
import numpy as np
X, idxContact = data.load_data(movie=405, min_ratings=200)
N = X.shape[1]

In [6]:
print(X.shape)

(943, 118)


In [7]:

# Creating and sparsifying the graph

nTotal = X.shape[0]
permutation = np.random.permutation(nTotal)
nTrain = int(np.ceil(0.9*nTotal))
idxTrain = permutation[0:nTrain]
nTest = nTotal-nTrain
idxTest=permutation[nTrain:nTotal]

W = data.create_graph(X=X, idxTrain=idxTrain, knn=40)

# Creating the training and test sets

xTrain, yTrain, xTest, yTest = data.split_data(X, idxTrain, idxTest, idxContact)
nTrain = xTrain.shape[0]
nTest = xTest.shape[0]

In [10]:
W

array([[0.        , 0.        , 0.        , ..., 0.0242602 , 0.        ,
        0.        ],
       [0.02562432, 0.        , 0.        , ..., 0.02433118, 0.        ,
        0.        ],
       [0.02596162, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02488369, 0.        , 0.        , ..., 0.        , 0.02698323,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02561502, 0.        ,
        0.        ],
       [0.02913293, 0.        , 0.02759221, ..., 0.        , 0.02910579,
        0.        ]])